# Coursera recomender System

## 1.Gathering Data from Coursera via request API

In [109]:
import pandas as pd
import requests

def get_courses_data():
    api_url = "https://api.coursera.org/api/courses.v1"
    params = {
        "start": 0,
        "limit": 10000,
        "fields": "id,name,slug,startDate,primaryLanguages,workload,isPartOfSpecialization"
    }
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        courses = data["elements"]
        
        course_data = []
        for course in courses:
            course_id = course["id"]
            course_name = course["name"]
            course_slug = course["slug"]
            start_date = course.get("startDate", "")
            end_date = course.get("endDate", "")
            primary_languages = course.get("primaryLanguages", [])
            workload = course.get("workload", "")
            primary_category = course.get("primaryCategory", "")
            instructor_ids = course.get("instructorIds", [])
            is_part_of_specialization = course.get("isPartOfSpecialization", False)
            skills = course.get("skills", [])
            audience = course.get("audience", "")
            course_format = course.get("courseFormat", "")
            
            course_data.append({
                "ID": course_id,
                "Name": course_name,
                "Slug": course_slug,
                "Start Date": start_date,
                "Primary Languages": primary_languages[0].split('-')[0],
                "Workload": workload,
                #"primary_category":primary_category,
                #"Instructor IDs": instructor_ids,
                "Is Part of Specialization": is_part_of_specialization,
            })
        
        df = pd.DataFrame(course_data)
        return df
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
        return None

courses_df = get_courses_data()
courses_df

,ID,Name,Slug,Start Date,Primary Languages,Workload,Is Part of Specialization
0,NJSdGN71Eeq4CApSN3OTvQ,Make Your Pick-Ups Look Cool in Unity (Intro t...,make-pick-ups-look-cool-unity-introduction-ani...,1598035773172,en,2 hours,False
1,DMkcgX7LEeyRTg6FtAvfBw,Intégrer des applications dans votre Dashboard...,integrer-applications-dashboard-hootsuite,1644614698077,fr,2 heures,False
2,YLO0oGSUEeyIUg4Qv2RsBQ,كيفية استعمال التطبيق هووتسويت,getting-started-with-hootsuite-ar,1642193091376,ar,2 hours,False
3,69Bku0KoEeWZtA4u62x6lQ,Gamification,gamification,1447095621493,en,4-8 hours/week,False
4,0HiU7Oe4EeWTAQ4yevf_oQ,Dealing With Missing Data,missing-data,1471901099812,en,"4 weeks of study, 1-2 hours/week",False
...,...,...,...,...,...,...,...
9995,ng-bUg0kEe2Y4woRe4onvw,Bildklassifizierung mit Tensorflow,bildklassifizierung-mit-tensorflow,1659393633053,de,1 Stunde,False
9996,oRWiSiO-EeuZCg52ZqAIxQ,الاستراتيجية التنافسية,competitive-strategy-ar,1619536132986,ar,10 ساعات فيديو واختبارات,False
9997,iClli1IjEey1wAq1DMRz-Q,Intro to Operating Systems 4: Persistence,codio-intro-to-operating-systems-4-persistence,1641316357195,en,,False
9998,a19TzyN0Eeu-rg7jvs9-1w,Bases de la planification et de la gestion de ...,uva-darden-project-management-fr,1618419430243,fr,"4 semaines d’études, 2-3 heures/semaine",False


In [114]:
courses_df['Start Date'] = pd.to_datetime(courses_df['Start Date'])
courses_df['Start Date'] = courses_df['Start Date'].dt.strftime('%d-%m-%Y %H:%M')
courses_df['Start Date'].describe()

count                10000
unique                   7
top       01-01-1970 00:27
freq                  4181
Name: Start Date, dtype: object

## 2.Database

### 2-1.Save to DB

In [8]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

def save_df_to_mysql(df, database, table_name, host='localhost', user='your_username', password='your_password'):
    # Create a MySQL connection
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password
    )
    
    # Create the database if it doesn't exist
    cursor = connection.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
    cursor.close()
    
    # Create SQLAlchemy engine
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    
    # Save the DataFrame to the MySQL table
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    
    # Close the connection
    connection.close()
    print(f"DataFrame saved to the '{database}.{table_name}' table.")

In [9]:
database_name = "DBLAB_final_project"
table_name = "test3"
save_df_to_mysql(courses_df, database_name, table_name, host='localhost', user='root', password='pdne')

DataFrame saved to the 'DBLAB_final_project.test3' table.


### 2-2.Connect to DB

In [1]:
import mysql
import mysql.connector
# run this every time
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="pdne",
 
)
mycursor = mydb.cursor()

mycursor.execute("SHOW DATABASES")
for db in mycursor:
    print(db)

('Coursera',)
('Coursera_DBLAB',)
('DB_Lab',)
('DBLAB_final_project',)
('information_schema',)
('myDatabase',)
('mysql',)
('performance_schema',)
('Session_Data',)
('Study',)
('sys',)
('TopLearn_Project',)


In [5]:
import pandas as pd
mycursor.execute("USE DBLAB_final_project")
courses_df = pd.read_sql('SELECT * FROM test3', mydb)
courses_df

/var/folders/bf/9616ns414yg2qn2p8czl2sr00000gn/T/ipykernel_18149/967033774.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  courses_df = pd.read_sql('SELECT * FROM test3', mydb)


,ID,Name,Slug,Start Date,Primary Languages,Workload,Is Part of Specialization
0,NJSdGN71Eeq4CApSN3OTvQ,Make Your Pick-Ups Look Cool in Unity (Intro t...,make-pick-ups-look-cool-unity-introduction-ani...,1598035773172,en,2 hours,0
1,DMkcgX7LEeyRTg6FtAvfBw,Intégrer des applications dans votre Dashboard...,integrer-applications-dashboard-hootsuite,1644614698077,fr,2 heures,0
2,YLO0oGSUEeyIUg4Qv2RsBQ,كيفية استعمال التطبيق هووتسويت,getting-started-with-hootsuite-ar,1642193091376,ar,2 hours,0
3,69Bku0KoEeWZtA4u62x6lQ,Gamification,gamification,1447095621493,en,4-8 hours/week,0
4,0HiU7Oe4EeWTAQ4yevf_oQ,Dealing With Missing Data,missing-data,1471901099812,en,"4 weeks of study, 1-2 hours/week",0
...,...,...,...,...,...,...,...
95,XXZBGc97EeufchLeGgZGZQ,Relational Database Administration (DBA),relational-database-administration,1637617419532,en,"5 weeks of study, 2-4 hours / week",0
96,M1lJ0BrjEeui5Apx-DHBkQ,Waits in Selenium Test Automation Tool,waits-in-selenium-test-automation-tool,1604094399504,en,2 hours,0
97,LZZg6vhQEeWfYgqbi1xsdw,Chinese Characters for beginner 汉字,hanzi,1477679733728,zh,5-7 hours/week,0
98,wFc3FhwIEeyf4Ar-aygVCw,Introducción a Hootsuite,introduccion-a-hootsuite,1633635891410,es,2 hours,0


## 3.Dataframe and Modification

## 4.GUI environment

In [7]:
import pandas as pd
import tkinter as tk
from tkinter import ttk

# Assuming you have the 'courses_df' DataFrame

# Create a Tkinter window
window = tk.Tk()

# Get the first 10 rows of the DataFrame
first_10_rows = courses_df.head(10)

# Create a Treeview widget to display the table
table = ttk.Treeview(window)

# Configure the table columns based on the DataFrame columns
table["columns"] = list(first_10_rows.columns)
for column in first_10_rows.columns:
    table.column(column, width=100)
    table.heading(column, text=column)

# Insert the data rows into the table
for _, row in first_10_rows.iterrows():
    table.insert("", "end", values=tuple(row))

# Place the table in the window
table.pack()

# Start the Tkinter event loop
window.mainloop()
